# House Prices: Advanced Regression Techniques

## 1. Getting data with Pandas

In [29]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

In [30]:
train_data = pd.read_csv('all/train.csv')
test_data  = pd.read_csv('all/test.csv')

In [31]:
train_label = train_data['SalePrice']
train_data_input = train_data.drop('SalePrice',1)
all_data   = pd.concat([train_data_input, test_data], axis=0)

print('Data Shape:\n')
print('Train Data \t\t:', train_data.shape)
print('Train Data(Input)\t:',train_data_input.shape)
print('Test  Data(Input)\t:',test_data.shape)
print('All Data \t\t:',all_data.shape)

(1460, 81)
(1460, 80)
(1459, 80)
(2919, 80)


## 2. Data exploration

In [32]:
all_data.describe()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [34]:
train_data['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [35]:
missing_data = all_data.isnull().sum().sort_values(ascending=False)
missing_data = missing_data[missing_data > 0]

In [36]:
count        = all_data.isnull().sum().sort_values(ascending=False)
percent_relt = all_data.isnull().sum() / all_data.isnull().count()
percent_relt = percent_relt.sort_values(ascending=False)*100

missing_data_perc  = pd.concat([count, percent_relt], axis=1, keys = ['Count', 'Percent(%)'])
missing_data_perc.head(50)

,Count,Percent(%)
PoolQC,2909,99.657417
MiscFeature,2814,96.402878
Alley,2721,93.216855
Fence,2348,80.438506
FireplaceQu,1420,48.646797
LotFrontage,486,16.649538
GarageCond,159,5.447071
GarageQual,159,5.447071
GarageYrBlt,159,5.447071
GarageFinish,159,5.447071


## 3. Prepare data

## 3.1 Drop Missing Data

In [37]:
all_data = all_data.drop((missing_data_perc[missing_data_perc['Percent(%)'] > 25]).index, 1)

all_data = all_data.drop('LotArea', 1)
all_data = all_data.drop('YearBuilt', 1)
all_data = all_data.drop('BsmtFinSF1', 1)

all_data = all_data.drop('BsmtFinSF2', 1)
all_data = all_data.drop('BsmtUnfSF', 1)
all_data = all_data.drop('TotalBsmtSF', 1)

all_data = all_data.drop('GarageArea', 1)
all_data = all_data.drop('WoodDeckSF', 1)
all_data = all_data.drop('OpenPorchSF', 1)
all_data = all_data.drop('EnclosedPorch', 1)
all_data = all_data.drop('3SsnPorch', 1)
all_data = all_data.drop('ScreenPorch', 1)

## 3.2 Fill Missing Data

In [38]:
all_data['LotFrontage']  = all_data['LotFrontage'].fillna(0)
all_data['GarageCond']   = all_data['GarageCond'].fillna('NA')
all_data['GarageType']   = all_data['GarageType'].fillna('NA')
all_data['GarageFinish'] = all_data['GarageFinish'].fillna('NA')
all_data['GarageQual']   = all_data['GarageQual'].fillna('NA')
all_data['BsmtExposure'] = all_data['BsmtExposure'].fillna('NA')
all_data['BsmtFinType2'] = all_data['BsmtFinType2'].fillna('NA')
all_data['BsmtFinType1'] = all_data['BsmtFinType1'].fillna('NA')
all_data['BsmtCond']     = all_data['BsmtCond'].fillna('NA')
all_data['BsmtQual']     = all_data['BsmtQual'].fillna('NA')
all_data['MasVnrArea']   = all_data['MasVnrArea'].fillna(0)
all_data['MasVnrType']   = all_data['MasVnrType'].fillna('None')
all_data['Electrical']   = all_data['Electrical'].fillna('SBrkr')
all_data['MSZoning']     = all_data['MSZoning'].fillna('RM')
all_data['Functional']   = all_data['Functional'].fillna('Typ')
all_data['BsmtFullBath'] = all_data['BsmtFullBath'].fillna(0)
all_data['BsmtHalfBath'] = all_data['BsmtFullBath'].fillna(0)
all_data['Utilities']    = all_data['Utilities'].fillna('AllPub')
#all_data['GarageArea']   = all_data['GarageArea'].fillna(0)
#all_data['BsmtFinSF2']   = all_data['BsmtFinSF2'].fillna(0)
#all_data['BsmtUnfSF']    = all_data['BsmtUnfSF'].fillna(0)
all_data['SaleType']     = all_data['SaleType'].fillna('Oth')
all_data['Exterior2nd']  = all_data['Exterior2nd'].fillna('Other')
all_data['Exterior1st']  = all_data['Exterior1st'].fillna('Other')
all_data['KitchenQual']  = all_data['KitchenQual'].fillna('TA')
all_data['GarageCars']   = all_data['GarageCars'].fillna(0)
#all_data['TotalBsmtSF']  = all_data['TotalBsmtSF'].fillna(0)

In [39]:
all_data['GarageYrBlt'].describe()
all_data['GarageYrBlt']  = all_data['GarageYrBlt'].fillna(1978)

In [40]:
print('MiscVal  == 0 percent -> %3.2f%%'%(100*(all_data['MiscVal'].values == 0).sum() / all_data['MiscVal'].values.shape[0] ))
print('PoolArea == 0 percent -> %3.2f%%'%(100*(all_data['PoolArea'].values == 0).sum() / all_data['PoolArea'].values.shape[0] ))

MiscVal  == 0 percent -> 96.47%
PoolArea == 0 percent -> 99.55%


In [41]:
all_data = all_data.drop('MiscVal', 1)
all_data = all_data.drop('PoolArea', 1)

print(all_data.isnull().sum().max())

0


In [42]:
numerical_data    = [var for var in all_data.columns if all_data.dtypes[var] != 'object']
categorical_data  = [var for var in all_data.columns if all_data.dtypes[var] == 'object']

numerical_data.remove('Id')
#numerical_data.remove('SalePrice')
print('Number of numerical data   : ', len(numerical_data))
print('Number of categorical data : ', len(categorical_data))

Number of numerical data   :  22
Number of categorical data :  38


## 3.3 Standarize Data

In [43]:
all_data = pd.get_dummies(all_data, columns=categorical_data)
scalers  = []
#all_data = all_data.astype(np.float32)
for name in all_data:
    if name == 'Id':
        continue
    x = all_data[name].values.astype(np.float32)
    x = x.reshape(-1,1)
    scaler = StandardScaler()
    scaler.fit(x)
    #scalers.append(scaler)
    all_data[name] = scaler.transform(x)

In [44]:
all_data.head()

,Id,MSSubClass,LotFrontage,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,0.067331,0.216075,0.646183,-0.507284,0.896833,0.529034,-0.773861,1.207379,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.302693
1,2,-0.873616,0.664158,-0.063185,2.188279,-0.395604,-0.567016,0.261075,-0.785025,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.302693
2,3,0.067331,0.305692,0.646183,-0.507284,0.848965,0.338903,-0.610718,1.235375,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.302693
3,4,0.302568,0.066714,0.646183,-0.507284,-0.682812,-0.567016,-0.506205,0.978742,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,3.789876,-0.064249,-0.09105,-0.126535,-2.155466,-0.302693
4,5,0.067331,0.783647,1.355551,-0.507284,0.753229,1.390216,-0.037170,1.671651,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.302693


## 3.4 Get Train Data and Test Data

In [45]:
train_data = all_data[:1460]
test_data = all_data[1460:]

## 4. Model

In [46]:
def getPrediction(W, b, input_data):
    return np.sum(W*input_data) + b

def RMSE(Y, pred, m):
    return np.sqrt(np.mean(np.square(np.log(Y+1)-np.log(pred+1))))

def train(input_, err, lr):
    global W, b
    W = W  + lr * err * input_
    b = b  + lr * err    

## 5. Training

In [47]:
train_data_values = train_data.values

In [48]:
train_input = train_data_values[:, 1:]
#train_label = train_data_values[:,-1]
train_label_values = train_label.values
train_input = train_input.astype(np.float64)

In [49]:
n, m = train_input.shape
regr = ElasticNet(random_state=0, max_iter = 1000)
regr.fit(train_input, train_label)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=0, selection='cyclic', tol=0.0001, warm_start=False)

In [50]:
pred_ar = regr.predict(train_input)
RMSE(train_label, pred_ar, m)

0.11855898672037692

## 6. Testing

In [51]:
test_data  = all_data[1460:]
test_data.head()


test_input = test_data.values.astype(np.float64)[:,1:]
#test_input       = test_data_values[:, 1:]
test_input       = test_input.astype(np.float32)

In [52]:
train_data.head()

,Id,MSSubClass,LotFrontage,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,0.067331,0.216075,0.646183,-0.507284,0.896833,0.529034,-0.773861,1.207379,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.302693
1,2,-0.873616,0.664158,-0.063185,2.188279,-0.395604,-0.567016,0.261075,-0.785025,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.302693
2,3,0.067331,0.305692,0.646183,-0.507284,0.848965,0.338903,-0.610718,1.235375,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.302693
3,4,0.302568,0.066714,0.646183,-0.507284,-0.682812,-0.567016,-0.506205,0.978742,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,3.789876,-0.064249,-0.09105,-0.126535,-2.155466,-0.302693
4,5,0.067331,0.783647,1.355551,-0.507284,0.753229,1.390216,-0.037170,1.671651,-0.101197,...,-0.052423,-0.298629,-0.052423,0.395018,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.302693


In [53]:
n, m         = test_input.shape
pred_test_ar = regr.predict(test_input)
#pred_test_ar = np.matmul(test_input,W) + b

In [54]:
pred_pd = pd.DataFrame(pred_test_ar, columns=['SalePrice'])
out     = pd.concat([test_data['Id'],pred_pd], axis=1)

In [55]:
out.to_csv('all/elastic_net.csv',index=False)
tt1 = pd.read_csv('all/elastic_net.csv')

In [56]:
tt1

,Id,SalePrice
0,1461,100395.051311
1,1462,150341.473322
2,1463,176850.360104
3,1464,190195.226140
4,1465,197906.556261
5,1466,171913.427358
6,1467,167364.949261
7,1468,168097.708153
8,1469,193632.361274
9,1470,118810.515728
